In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from math import sqrt
import xgboost

from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
%matplotlib inline

In [10]:
train_df = pd.read_csv("train.csv", low_memory=False)
test_df = pd.read_csv("test.csv", low_memory=False)
data = pd.read_csv("data_set.csv", low_memory = False)

In [11]:
train_df = train_df[['id','precio']]
train_df['precio'].value_counts()

1500000.0    3041
2500000.0    3036
1200000.0    2909
3500000.0    2726
1300000.0    2480
             ... 
2704600.0       1
1352250.0       1
938260.0        1
2704460.0       1
2327000.0       1
Name: precio, Length: 15520, dtype: int64

In [12]:
data = data[['id','days_to_today',
 'antiguedad',
 'habitaciones',
 'garages',
 'banos',
 'Apartamento',
 'Casa',
 'Edificio',
 'Terreno',
 'gimnasio',
 'qty_of_extras',
 'metroscubiertos',
 'metrostotales',
 'popular_desc_1',
 'popular_desc_2',
 'popular_desc_7',
 'popular_desc_13',
 'idzona',
 'lat',
 'lng',
 'Coahuila',
 'Distrito Federal',
 'Edo. de México',
 'Jalisco',
 'Nuevo León',
 'Yucatán',
 'dolar',
 'habxmetros',
 'habxmetroscub',
 'banosxmetroscub',
 'banosxhab']]
data.shape

(300000, 32)

In [13]:
test_df.shape

(60000, 22)

In [14]:
train_df.shape

(240000, 2)

In [15]:
features = pd.merge(train_df, data, on='id', how='inner')
features = features.fillna(0)
features['precio'].value_counts()

1500000.0    3041
2500000.0    3036
1200000.0    2909
3500000.0    2726
1300000.0    2480
             ... 
2704600.0       1
1352250.0       1
938260.0        1
2704460.0       1
2327000.0       1
Name: precio, Length: 15520, dtype: int64

In [16]:
labels = features['precio']
features = features.drop(['id','precio'], axis=1)


In [17]:
set_test = pd.merge(data, test_df[['id']],on = "id", how = "inner")
ids = set_test["id"]
set_test = set_test.drop(columns=['id'])

In [18]:
set_test = set_test.fillna(0)

In [19]:
set_test.head()

,days_to_today,antiguedad,habitaciones,garages,banos,Apartamento,Casa,Edificio,Terreno,gimnasio,...,Distrito Federal,Edo. de México,Jalisco,Nuevo León,Yucatán,dolar,habxmetros,habxmetroscub,banosxmetroscub,banosxhab
0,2324,29.0,3.0,0.0,4.0,0,1,0,0,0.0,...,1,0,0,0,0,12.7325,0.000000,0.010000,0.013333,0.0
1,1498,0.0,1.0,1.0,1.0,1,0,0,0,0.0,...,0,0,0,0,1,16.5000,0.014925,0.014925,0.014925,1.0
2,1645,0.0,2.0,1.0,2.0,1,0,0,0,0.0,...,1,0,0,0,0,15.3775,0.020000,0.022989,0.022989,2.0
3,1703,2.0,2.0,2.0,2.0,1,0,0,0,0.0,...,0,0,0,0,0,15.3488,0.023256,0.023256,0.023256,1.0
4,2298,10.0,2.0,1.0,1.0,0,1,0,0,0.0,...,0,1,0,0,0,13.3803,0.026316,0.025000,0.012500,2.0


In [20]:
regressor = xgboost.XGBRegressor(objective='reg:squarederror', n_estimators=500, learning_rate=0.02,
                             gamma=0, subsample=0.5, colsample_bytree=0.5, max_depth=10, min_child_weight=1)

regressor.fit(features, labels)

/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/usr/local/lib/python3.6/dist-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.5, gamma=0,
             importance_type='gain', learning_rate=0.02, max_delta_step=0,
             max_depth=10, min_child_weight=1, missing=None, n_estimators=500,
             n_jobs=1, nthread=None, objective='reg:squarederror',
             random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
             seed=None, silent=None, subsample=0.5, verbosity=1)

In [21]:
preds_kaggel_knn = regressor.predict(set_test)

In [22]:
df_kaggel_knn = pd.DataFrame(preds_kaggel_knn)
df_kaggel_knn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60000 entries, 0 to 59999
Data columns (total 1 columns):
0    60000 non-null float32
dtypes: float32(1)
memory usage: 234.5 KB


In [23]:
df_kaggel_knn = pd.DataFrame(df_kaggel_knn)

In [24]:
df_kaggel_knn.head()

,0
0,6.784103e+06
1,1.133810e+06
2,2.421022e+06
3,1.392149e+06
4,6.405852e+05


In [25]:
# df_kaggel_knn = df_kaggel_knn.to_frame()# df_kagg 
df_kaggel_knn["id"] = ids
df_kaggel_knn.rename(columns = {0: 'target'},inplace = True) 
df_kaggel_knn = df_kaggel_knn[['id','target']]

In [26]:
df_kaggel_knn.head()

,id,target
0,4941,6.784103e+06
1,51775,1.133810e+06
2,115253,2.421022e+06
3,299321,1.392149e+06
4,173570,6.405852e+05


In [27]:
df_kaggel_knn.to_csv('ResultadosXGB-final.csv',index=False)

In [28]:
df_kaggel_knn.shape

(60000, 2)

In [29]:
df_kaggel_knn.head()

,id,target
0,4941,6.784103e+06
1,51775,1.133810e+06
2,115253,2.421022e+06
3,299321,1.392149e+06
4,173570,6.405852e+05
